In [3]:
import os
import numpy as np
import pandas as pd
import warnings

from google.colab import drive

warnings.filterwarnings('ignore')
drive.mount("/content/drive")

os.chdir("drive/MyDrive/AI competition/Round 1")
os.listdir()

Mounted at /content/drive


['competition_data',
 '__pycache__',
 'html',
 'submission',
 'logs.log',
 'playground.ipynb',
 'AutoML Baseline.ipynb',
 'Evaluator Module.ipynb',
 'evaluator.py',
 '2022-08-04 feature EDA.ipynb',
 "2022-08-05 LGB_train(hyeonbin's parameter tuning).ipynb",
 'model compare.ipynb',
 'Optuna Optimization.ipynb']

In [4]:
from evaluator import Model, Evaluator

train_df = pd.read_csv('competition_data/train.csv')
test_df = pd.read_csv("competition_data/test.csv")
submission_df = pd.read_csv("competition_data/sample_submission.csv")
train_df.head()

,index,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,engnat,age,hand,religion,orientation,voted,married,familysize,ASD,nerdiness
0,0,1.0,5.0,5.0,5.0,1.0,4.0,5.0,5.0,1.0,...,1.0,20,2.0,12.0,4.0,2.0,1.0,4.0,2.0,1
1,1,4.0,4.0,4.0,4.0,4.0,5.0,4.0,4.0,3.0,...,1.0,49,1.0,2.0,1.0,1.0,2.0,4.0,2.0,1
2,2,4.0,5.0,5.0,4.0,3.0,5.0,5.0,5.0,4.0,...,2.0,43,1.0,2.0,2.0,2.0,3.0,4.0,2.0,1
3,3,4.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0,3.0,...,1.0,17,2.0,1.0,1.0,2.0,1.0,2.0,2.0,1
4,4,4.0,4.0,4.0,4.0,3.0,3.0,4.0,2.0,3.0,...,2.0,18,2.0,12.0,1.0,2.0,1.0,1.0,2.0,0


## Baseline

In [5]:
params = {
    "num_leaves":31, 
    "max_depth":-1, 
    "learning_rate":0.1, 
    "n_estimators":100, 
    "subsample_for_bin":200000, 
    "objective":"binary", 
    "class_weight":None, 
    "min_split_gain":0.0, 
    "min_child_weight":0.001, 
    "min_child_samples":20, 
    "subsample":1.0, 
    "subsample_freq":0, 
    "colsample_bytree":1.0, 
    "reg_alpha":0.0, 
    "reg_lambda":0.0,
}

Evaluator(
    model=Model("lgbm", "clf", **params).get_model(),
    train_df=train_df
).run()

0it [00:00, ?it/s]

,accuracy,precision,recall,f1-score,roc_auc,mae,train_acc
fold,,,,,,,
1,0.751485,0.758924,0.816815,0.786806,0.742335,0.248515,NaN
2,0.749297,0.745180,0.820425,0.780994,0.742286,0.250703,NaN
3,0.745233,0.752886,0.806633,0.778833,0.737471,0.254767,NaN
4,0.750234,0.775844,0.797290,0.786421,0.741703,0.249766,NaN
mean,0.749062,0.758208,0.810291,0.783263,0.740949,0.250938,0.853704


# Optuna

## 1st round

In [ ]:
# !pip install optuna

In [8]:
import optuna

In [9]:
def objective(trial):
    param = {
        'objective': 'binary', 
        "n_estimators": 100,
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1e-1),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1e-1),
        "num_leaves": trial.suggest_int("num_leaves", 20, 100),
        # 'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-2, 1),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    result_df = Evaluator(
        model=Model("lgbm", "clf", **param).get_model(),
        train_df=train_df
    ).run(train_acc=False)

    return result_df["roc_auc"]["mean"]

In [10]:
opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=42))
opt.optimize(objective, n_trials=100)

[I 2022-08-06 02:19:22,099] A new study created in memory with name: no-name-74e4b1b6-e132-453b-a6be-7d034d3212d2


0it [00:00, ?it/s]

[I 2022-08-06 02:19:29,963] Trial 0 finished with value: 0.7697556967115567 and parameters: {'reg_alpha': 4.185822729546961e-06, 'reg_lambda': 0.04518560951024101, 'num_leaves': 79, 'learning_rate': 0.15751320499779725, 'min_child_samples': 19}. Best is trial 0 with value: 0.7697556967115567.


0it [00:00, ?it/s]

[I 2022-08-06 02:19:35,139] Trial 1 finished with value: 0.7702188499894239 and parameters: {'reg_alpha': 1.23583827723069e-07, 'reg_lambda': 2.5502648504032812e-08, 'num_leaves': 90, 'learning_rate': 0.15930522616241014, 'min_child_samples': 72}. Best is trial 1 with value: 0.7702188499894239.


...

[I 2022-08-06 02:26:34,676] Trial 97 finished with value: 0.7460441194072912 and parameters: {'reg_alpha': 0.02032853460767353, 'reg_lambda': 2.0802937409093856e-08, 'num_leaves': 42, 'learning_rate': 0.7958348774085425, 'min_child_samples': 90}. Best is trial 51 with value: 0.7719294728933701.


0it [00:00, ?it/s]

[I 2022-08-06 02:26:39,081] Trial 98 finished with value: 0.7268186397393659 and parameters: {'reg_alpha': 1.547381034082918e-05, 'reg_lambda': 0.00021924423608141497, 'num_leaves': 42, 'learning_rate': 0.023781668412074273, 'min_child_samples': 49}. Best is trial 51 with value: 0.7719294728933701.


0it [00:00, ?it/s]

[I 2022-08-06 02:26:42,184] Trial 99 finished with value: 0.7397451839149812 and parameters: {'reg_alpha': 2.974853588434834e-06, 'reg_lambda': 0.00012178471467125296, 'num_leaves': 26, 'learning_rate': 0.888770467723736, 'min_child_samples': 99}. Best is trial 51 with value: 0.7719294728933701.


In [11]:
opt.best_trial.value

0.7719294728933701

In [12]:
opt.best_trial.params

{'learning_rate': 0.13807731717915697,
 'min_child_samples': 14,
 'num_leaves': 76,
 'reg_alpha': 7.899667957519268e-05,
 'reg_lambda': 0.03573414695793331}

### Analysis

In [13]:
optuna.visualization.plot_param_importances(opt)

In [14]:
optuna.visualization.plot_optimization_history(opt)

In [15]:
optuna.visualization.plot_slice(opt)

## 2nd round

In [41]:
def objective(trial):
    param = {
        'objective': 'binary', 
        "n_estimators": 200,
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1e-1),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1e-1),
        "num_leaves": trial.suggest_int("num_leaves", 50, 500),
        # 'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-2, 2e-1),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    result_df = Evaluator(
        model=Model("lgbm", "clf", **param).get_model(),
        train_df=train_df
    ).run(train_acc=False)

    return result_df["roc_auc"]["mean"]

opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=42))
opt.optimize(objective, n_trials=100)

[I 2022-08-05 13:35:27,707] A new study created in memory with name: no-name-9fc73d9f-f840-4d6e-981e-5abe9dc108fe


0it [00:00, ?it/s]

[I 2022-08-05 13:35:50,093] Trial 0 finished with value: 0.772757411191133 and parameters: {'reg_alpha': 4.185822729546961e-06, 'reg_lambda': 0.04518560951024101, 'num_leaves': 380, 'learning_rate': 0.060099747183803134}. Best is trial 0 with value: 0.772757411191133.


0it [00:00, ?it/s]

[I 2022-08-05 13:35:58,757] Trial 1 finished with value: 0.7692450673939745 and parameters: {'reg_alpha': 1.2363188277052202e-07, 'reg_lambda': 1.23583827723069e-07, 'num_leaves': 76, 'learning_rate': 0.13394334706750485}. Best is trial 0 with value: 0.772757411191133.


...

[I 2022-08-05 14:05:46,264] Trial 97 finished with value: 0.7718524635640217 and parameters: {'reg_alpha': 0.001795987300739819, 'reg_lambda': 0.0044447073867903775, 'num_leaves': 496, 'learning_rate': 0.034421355953493375}. Best is trial 71 with value: 0.7754921200677745.


0it [00:00, ?it/s]

[I 2022-08-05 14:06:03,038] Trial 98 finished with value: 0.7742019126078037 and parameters: {'reg_alpha': 4.019079533739863e-06, 'reg_lambda': 0.0027220163230221046, 'num_leaves': 203, 'learning_rate': 0.16253361666370852}. Best is trial 71 with value: 0.7754921200677745.


0it [00:00, ?it/s]

[I 2022-08-05 14:06:24,477] Trial 99 finished with value: 0.7748795110357147 and parameters: {'reg_alpha': 0.01020679196117236, 'reg_lambda': 1.0068347388834027e-05, 'num_leaves': 388, 'learning_rate': 0.09587004080044081}. Best is trial 71 with value: 0.7754921200677745.


In [42]:
print(opt.best_trial.value)
opt.best_trial.params

0.7754921200677745


{'learning_rate': 0.050815587001436395,
 'num_leaves': 259,
 'reg_alpha': 0.00011167629235521655,
 'reg_lambda': 1.784826134719029e-08}

### analysis

In [43]:
optuna.visualization.plot_param_importances(opt)

In [44]:
optuna.visualization.plot_optimization_history(opt)

In [45]:
optuna.visualization.plot_slice(opt)

## 3rd Round

In [46]:
def objective(trial):
    param = {
        'objective': 'binary', 
        "n_estimators": 500,
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-8, 1e-1),
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-8, 1e-1),
        "num_leaves": trial.suggest_int("num_leaves", 200, 1000),
        # 'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-2, 2e-1),
        # 'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    result_df = Evaluator(
        model=Model("lgbm", "clf", **param).get_model(),
        train_df=train_df
    ).run(train_acc=False)

    return result_df["roc_auc"]["mean"]

opt = optuna.create_study(direction='maximize',sampler=optuna.samplers.RandomSampler(seed=42))
opt.optimize(objective, n_trials=100)

[I 2022-08-05 14:11:03,825] A new study created in memory with name: no-name-a532e340-b97c-42f6-b8a8-c7de01ad2bdc


0it [00:00, ?it/s]

[I 2022-08-05 14:12:06,965] Trial 0 finished with value: 0.7700659607537392 and parameters: {'reg_alpha': 4.185822729546961e-06, 'reg_lambda': 0.04518560951024101, 'num_leaves': 786, 'learning_rate': 0.060099747183803134}. Best is trial 0 with value: 0.7700659607537392.


0it [00:00, ?it/s]

[I 2022-08-05 14:12:46,315] Trial 1 finished with value: 0.7707096057835258 and parameters: {'reg_alpha': 1.2363188277052202e-07, 'reg_lambda': 1.23583827723069e-07, 'num_leaves': 246, 'learning_rate': 0.13394334706750485}. Best is trial 1 with value: 0.7707096057835258.


...

[I 2022-08-05 15:35:05,456] Trial 97 finished with value: 0.7749760309058495 and parameters: {'reg_alpha': 0.001795987300739819, 'reg_lambda': 0.0044447073867903775, 'num_leaves': 993, 'learning_rate': 0.034421355953493375}. Best is trial 33 with value: 0.7783073958853803.


0it [00:00, ?it/s]

[I 2022-08-05 15:35:46,445] Trial 98 finished with value: 0.7735060046784388 and parameters: {'reg_alpha': 4.019079533739863e-06, 'reg_lambda': 0.0027220163230221046, 'num_leaves': 472, 'learning_rate': 0.16253361666370852}. Best is trial 33 with value: 0.7783073958853803.


0it [00:00, ?it/s]

[I 2022-08-05 15:36:26,420] Trial 99 finished with value: 0.7744198347986002 and parameters: {'reg_alpha': 0.01020679196117236, 'reg_lambda': 1.0068347388834027e-05, 'num_leaves': 801, 'learning_rate': 0.09587004080044081}. Best is trial 33 with value: 0.7783073958853803.


In [47]:
print(opt.best_trial.value)
opt.best_trial.params

0.7783073958853803


{'learning_rate': 0.026332779906149555,
 'num_leaves': 955,
 'reg_alpha': 6.90331310095056e-08,
 'reg_lambda': 2.30837413695962e-06}

In [48]:
optuna.visualization.plot_param_importances(opt)

In [49]:
optuna.visualization.plot_optimization_history(opt)

In [50]:
optuna.visualization.plot_slice(opt)